<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1>Solicitudes
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.colombiacompra.gov.co/sites/cce_public/files/logo_dnp.jpg" style="width: 200px;"></a></div>
</div>

# Librerias

In [1]:
! pip install sodapy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import os
from sodapy import Socrata
import json

# Conexión con las bases de SECOP I y SECOP II

In [12]:
# SECOP I
client = Socrata("www.datos.gov.co", "z0stE9vAoMEFJLPgelwqYg9TP",timeout=200000)
secop_I_identifier = "f789-7hwg"
dataset_metadata_secop_I = client.get_metadata(secop_I_identifier)

# SECOP II
secop_II_identifier = "jbjy-vk9h"
dataset_metadata_secop_II = client.get_metadata(secop_II_identifier)

In [21]:
query = """select *"""
# query += """ where anno_firma_contrato >= '2018'"""   # Filtro por fecha
# query += """ and detalle_del_objeto_a_contratar like '%solidar%' or"""  # Filtro por palabra clave
# query += """ detalle_del_objeto_a_contratar like '%ong%' """  # Filtro por palabra clave
query += """ limit 100"""  # Limite de datos a descargar

results_secop_I = client.get(secop_I_identifier,content_type="json",query=query)
results_secop_I_df = pd.DataFrame.from_records(results_secop_I)

results_secop_II = client.get(secop_II_identifier,content_type="json",query=query)
results_secop_II_df = pd.DataFrame.from_records(results_secop_II)

# Unificación de bases

In [22]:
HC=pd.read_csv('..\..\..\Sample_Data\Raw\Homologa_columnas.csv')

In [23]:
HC=HC[HC['API_DA_S_I'].notna()]
HC=HC[HC['API_DA_S_II'].notna()]

results_secop_I_df['fuente']='SECOP I'
results_secop_I_df=results_secop_I_df[HC['API_DA_S_I']]

results_secop_II_df['fuente']='SECOP II'
results_secop_II_df=results_secop_II_df[HC['API_DA_S_II']]

results_secop_I_df.columns=HC['Unificado']
results_secop_II_df.columns=HC['Unificado']

In [24]:
Consulta=pd.concat([results_secop_I_df,results_secop_II_df],axis=0)

In [25]:
Consulta.head()

Unificado,Fuente,ID Contrato,ID Proceso,ID entidad Plataforma,Entidad,NIT Entidad,Orden,Modalidad,Estado,Descripción proceso,...,UNSPSC,Nombre Proveedor,Documento Proveedor,Tipo de documento proveedor,Valor del contrato,Departamento Entidad,Ciudad Entidad,Inicio de contrato,Fin de contrato,URL
0,SECOP I,10209031382021,21-12-11830159-10958055,199000003,ESCUELA NACIONAL DEL DEPORTE,805001868,TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Celebrado,Prestación de servicios profesionales en la di...,...,801116,WILFREDO AGREDO RODRIGUEZ,16639187,Cédula de Ciudadanía,22500000,Bogotá DC,Bogotá DC,2021-03-25T00:00:00.000,2021-07-30T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...
1,SECOP I,26762019,19-12-9752637-8884899,133001000,MINISTERIO DE CULTURA MINCULTURA,830034348,NACIONAL,Contratación Directa (Ley 1150 de 2007),Celebrado,El objeto del presente contrato es Realizar el...,...,801116,LUIS MANUEL REALES MAESTRE,79955263,Cédula de Ciudadanía,23430330,Bogotá DC,Bogotá DC,2019-07-24T00:00:00.000,2019-12-25T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...
2,SECOP I,SCRD0082015,15-4-3450708-3255179,276000012,VALLE DEL CAUCA UNIVERSIDAD DEL VALLE,890399010,TERRITORIAL,Régimen Especial,Celebrado,COMPRA DE POLVO DE LADRILLO TIERRA ROJA CAL PA...,...,N/D,SERVICANCHAS Y PISICINA EU,900434754,Nit de Persona Jurídica,8491200,Valle del Cauca,Cali,2015-02-02T00:00:00.000,2015-03-04T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...
3,SECOP I,mccd0612017,17-12-6903211-6272322,215212011,BOYACÁ ALCALDÍA MUNICIPIO DE COPER,890980767-3,TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Celebrado,PRESTACIÓN DE SERVICIOS DE APOYO LOGÍSTICO EN ...,...,801116,ADRIANA TORRES SANCHEZ,23444192,Cédula de Ciudadanía,9000000,Boyacá,Coper,2017-07-13T00:00:00.000,2017-08-15T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...
4,SECOP I,No definido,15-12-3642523,276001722,CUNDINAMARCA HOSPITAL DIVINO SALVADOR DE SOPO,860023878,TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Convocado,PRESTACIÓN DE SERVICIOS DE AUXILIAR DE ESTADÍS...,...,N/D,No Definido,No Definido,No Definido,0,Cundinamarca,Sopó,NaN,NaN,{'url': 'https://www.contratos.gov.co/consulta...


In [26]:
Consulta.to_csv('..\..\..\Sample_Data\Raw\Consulta_total.csv',index=False)

In [27]:
Consulta.shape

(200, 23)